In [12]:
import pandas as pd
import numpy as np

# Load Titanic dataset
df = pd.read_csv("titanic.csv")

# Select relevant features
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]
label = "Survived"

# Fill missing values
df["Age"].fillna(df["Age"].median(), inplace=True)
df["Fare"].fillna(df["Fare"].median(), inplace=True)

# Convert categorical variables
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})

# Normalize numerical columns (Min-Max Scaling)
for col in ["Age", "Fare"]:
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

# Extract features and labels
X = df[features].values  # Convert to NumPy array
y = df[label].values.reshape(-1, 1)  # Ensure it's a column vector

In [ ]:
# Sigmoid activation function
def sigmoid(x):
    x = np.clip(x, -500, 500)  # Prevent extreme values
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Initialize network with random weights and biases
def initialize_network(input_size, hidden_size, output_size):
    return {
        "W1": np.random.uniform(-1, 1, (input_size, hidden_size)),  # (input_size, hidden_size)
        "B1": np.random.uniform(-1, 1, (1, hidden_size)),           # (1, hidden_size)
        "W2": np.random.uniform(-1, 1, (hidden_size, output_size)), # (hidden_size, output_size)
        "B2": np.random.uniform(-1, 1, (1, output_size))            # (1, output_size)
    }

# Forward pass
def forward_pass(network, inputs):
    hidden_layer_input = np.dot(inputs, network["W1"]) + network["B1"]
    hidden_layer_output = sigmoid(hidden_layer_input)

    output_layer_input = np.dot(hidden_layer_output, network["W2"]) + network["B2"]
    output_layer_output = sigmoid(output_layer_input)

    return hidden_layer_input, hidden_layer_output, output_layer_input, output_layer_output

# Backpropagation
def backward_pass(network, inputs, hidden_layer_input, hidden_output, output_output, expected, learning_rate):
    output_errors = expected - output_output  # Error at output
    output_deltas = output_errors * sigmoid_derivative(output_output)  # Gradient at output

    hidden_errors = np.dot(output_deltas, network["W2"].T)  # Backpropagated error
    hidden_deltas = hidden_errors * sigmoid_derivative(hidden_output)  # Gradient at hidden layer

    # Update weights and biases
    network["W2"] += learning_rate * np.dot(hidden_output.T, output_deltas)
    network["B2"] += learning_rate * np.sum(output_deltas, axis=0, keepdims=True)

    network["W1"] += learning_rate * np.dot(inputs.T, hidden_deltas)
    network["B1"] += learning_rate * np.sum(hidden_deltas, axis=0, keepdims=True)

# Train the network
def train_network(network, X, y, epochs=1000, learning_rate=0.1):
    for epoch in range(epochs):
        hidden_layer_input, hidden_output, output_layer_input, output_output = forward_pass(network, X)
        backward_pass(network, X, hidden_layer_input, hidden_output, output_output, y, learning_rate)

        if epoch % 100 == 0:
            loss = np.mean((y - output_output) ** 2)  # MSE Loss
            accuracy = np.mean((output_output >= 0.5) == y)  # Binary classification accuracy
            print(f"Epoch {epoch}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}")

# Split dataset into training and testing
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Initialize and train
network = initialize_network(input_size=X.shape[1], hidden_size=10, output_size=1)
train_network(network, X_train, y_train, epochs=1000, learning_rate=0.01)

# Evaluate on test set
_, _, _, test_output = forward_pass(network, X_test)
test_accuracy = np.mean((test_output >= 0.5) == y_test)
print(f"\nTest Accuracy: {test_accuracy:.4f}")

Epoch 0: Loss = 0.2671, Accuracy = 0.3904
Epoch 100: Loss = 0.1439, Accuracy = 0.8076
Epoch 200: Loss = 0.1420, Accuracy = 0.8062
Epoch 300: Loss = 0.1405, Accuracy = 0.8034
Epoch 400: Loss = 0.1396, Accuracy = 0.8062
Epoch 500: Loss = 0.1390, Accuracy = 0.8048
Epoch 600: Loss = 0.1385, Accuracy = 0.8090
Epoch 700: Loss = 0.1381, Accuracy = 0.8104
Epoch 800: Loss = 0.1377, Accuracy = 0.8076
Epoch 900: Loss = 0.1373, Accuracy = 0.8090

Test Accuracy: 0.8492
